In [2]:
import torch
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, OPTForCausalLM, MistralForCausalLM

/home/dingfangyu/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = 'meta-llama/Llama-2-70b-hf'

model = AutoModelForCausalLM.from_pretrained(model_id)

Loading checkpoint shards: 100%|██████████| 15/15 [00:18<00:00,  1.22s/it]


In [4]:
W_K = model.model.layers[0].self_attn.k_proj.weight#.T
W_V = model.model.layers[0].self_attn.v_proj.weight#.T
X = torch.randn(100, 8192)

In [5]:
W_K_pinv = torch.linalg.pinv(W_K)

In [6]:
W_K_pinv.shape, W_K.shape

(torch.Size([8192, 1024]), torch.Size([1024, 8192]))

In [7]:
W_K_pinv @ W_K

tensor([[ 1.0306e-01, -1.0693e-04, -3.2222e-03,  ...,  5.3073e-03,
          4.3491e-04,  5.5801e-03],
        [-1.0745e-04,  7.9584e-02,  1.0892e-03,  ..., -2.2894e-03,
          2.9489e-03,  5.1781e-03],
        [-3.2223e-03,  1.0894e-03,  7.4302e-02,  ..., -1.7048e-03,
         -3.8307e-03, -2.8403e-04],
        ...,
        [ 5.3073e-03, -2.2893e-03, -1.7048e-03,  ...,  9.3425e-02,
          2.1545e-03,  4.0117e-03],
        [ 4.3507e-04,  2.9494e-03, -3.8303e-03,  ...,  2.1545e-03,
          1.6254e-01, -6.2004e-03],
        [ 5.5801e-03,  5.1784e-03, -2.8379e-04,  ...,  4.0117e-03,
         -6.2003e-03,  2.2664e-01]], grad_fn=<MmBackward0>)

In [8]:
W_K @ W_K_pinv 

tensor([[ 1.0000e+00, -4.7544e-07,  2.4214e-08,  ...,  2.2631e-07,
         -2.8871e-08, -9.5461e-09],
        [-4.9774e-06,  1.0000e+00,  2.4971e-07,  ..., -6.0536e-07,
         -8.2422e-07, -6.7241e-07],
        [ 1.8582e-06,  1.3039e-08,  1.0000e+00,  ..., -5.3551e-07,
         -1.8906e-07, -1.0128e-07],
        ...,
        [ 2.3049e-06, -1.6764e-08,  5.2294e-07,  ...,  1.0000e+00,
          2.6822e-07, -4.0233e-07],
        [-1.6261e-06,  1.0431e-07, -5.5367e-07,  ...,  1.1325e-06,
          1.0000e+00, -7.8231e-08],
        [-2.9895e-07,  1.6578e-07,  2.5216e-07,  ..., -1.4007e-06,
          3.0827e-07,  1.0000e+00]], grad_fn=<MmBackward0>)

In [10]:
torch.dist(W_K @ W_K_pinv @ W_V, W_V)

tensor(0.0004, grad_fn=<DistBackward0>)

In [11]:
torch.linalg.matrix_rank(W_K_pinv @ W_V)

tensor(1024)

In [12]:
(W_K_pinv @ W_V).shape

torch.Size([8192, 8192])

In [13]:
A = W_K_pinv @ W_V

In [18]:
torch.dist(W_K @ A @ X.T, W_V @ X.T)

tensor(0.0035, grad_fn=<DistBackward0>)

In [20]:
A_p = torch.linalg.pinv(A)